In [ ]:
from typing import List, Optional
import vertexai
import pandas as pd

from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

PROJECT_ID =
LOCATION = 
vertexai.init(project=PROJECT_ID, location=LOCATION)

def embed_text(
    texts: List[str],
    task: str = "RETRIEVAL_DOCUMENT",
    model_name: str = "text-embedding-004",
    dimensionality: Optional[int] = 256,
) -> List[List[float]]:
    """Embeds texts with a pre-trained, foundational model."""
    model = TextEmbeddingModel.from_pretrained(model_name)

    inputs = [TextEmbeddingInput(text, task) for text in texts]
    kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}
    embeddings = model.get_embeddings(inputs, **kwargs)
    return [embedding.values for embedding in embeddings]

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel


def llm_prediction(prompt:str):
    model = GenerativeModel(model_name="gemini-1.0-pro-002",generation_config=GenerationConfig(
                temperature=0.0,
                top_p=1,
                top_k=10,
                candidate_count=1,
                max_output_tokens=250,
                stop_sequences=["STOP!"],
            ),)
    
    context=retreive_context(prompt)
    prompt_text = f"""DOCUMENT:
    {context}

     QUESTION:
     {prompt}
     
     INSTRUCTIONS:
     Answer the users QUESTION using the DOCUMENT text above.
     Keep your answer ground in the facts of the DOCUMENT.
     If the DOCUMENT doesn’t contain the facts to answer the QUESTION return "I dont know the answer !" """
    
    return model.generate_content(prompt_text).text

In [ ]:
def retreive_context(question:str):
    request=[question]
    request_embedding = embed_text(request)
    results = collection.query(
        query_embeddings=request_embedding,
        n_results=3,    
    )
    context = "\n##".join([text for text in results['documents'][0]])
    return context

In [ ]:
question= "Explain the formula used for training the LLaVa model"
output = llm_prediction(question)